## KELOMPOK 3

- Kasyful Haq Bachariputra (2304820)
- Muhammad Akhtar Rizki Ramadha (2304742)
- Ibnu Fadhilah (2300613)
- Muhammad Naufal Arbanin (2310850)
- Hanif Ahmad Syauqi (2304330)

## PENDAHULUAN


Notebook ini adalah hasil analisis data transaksi yang melibatkan langkah-langkah pembersihan data, pengisian nilai yang hilang, dan penggunaan model Random Forest untuk memprediksi hasil. Berikut ini adalah rangkuman tahapan dalam notebook ini:

1. Persiapan Data: Menggunakan dataset latihan, data disiapkan dengan mengganti simbol "?" dengan nilai NaN dan mengisi nilai yang hilang menggunakan metode rata-rata (mean) untuk kolom numerik dan metode most frequent untuk kolom objek.

2. Pembuatan Model: Model Random Forest dikembangkan dengan membagi data menjadi data latih dan validasi, kemudian dilakukan penskalaan data. Model ini dievaluasi menggunakan matriks kebingungan dan laporan klasifikasi.

3. Prediksi Data Uji: Data uji juga diproses dan diprediksi menggunakan model yang telah dilatih.

In [20]:
# Melakukan import library
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Melakukan EDA pada dataset training
df_train = pd.read_csv("transact_train.txt", sep='|')

df_train = df_train.replace('?', np.nan) # Mengganti '?' dengan not a number, sehingga bisa dilakukan pembersihan data

# Menambah data pada kolom yang merupakan tipe data float atau int menggunakan mean
num = df_train.select_dtypes(include=['float64', 'int64']).columns
num_imputer = SimpleImputer(strategy='mean')
df_train[num] = num_imputer.fit_transform(df_train[num])

# Menambah data pada kolom yang merupakan tipe objek menggunakan 'most frequent'
obj = df_train.select_dtypes(include='object').columns
obj_imputer = SimpleImputer(strategy='most_frequent')
df_train[obj] = obj_imputer.fit_transform(df_train[obj])

## MODEL

In [21]:
# Menentukan feature mana saja yang akan digunakan dalam model
x = df_train.iloc[:, [2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 16, 18, 19, 20, 22]].values
# Menentukan target hasil prediksi
y = df_train.iloc[:, -1].values

# Melakuakn import library
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Membagi dua dataset menjadi data training dan data validation
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# Melakuakn scaling data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Menggunakan model RandomForest untuk data learning
classi = RandomForestClassifier(n_estimators=50, random_state=0)
classi.fit(x_train, y_train)

# Melakukan prediksi
y_pred = classi.predict(x_test)

# Membuat confusion matrix serta accuracy score untuk evaluasi
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
# Menampilkan hasil evaluasi
print(cm)
print(classification_report(y_test, y_pred))

[[23371  4386]
 [ 2646 55400]]
              precision    recall  f1-score   support

           n       0.90      0.84      0.87     27757
           y       0.93      0.95      0.94     58046

    accuracy                           0.92     85803
   macro avg       0.91      0.90      0.90     85803
weighted avg       0.92      0.92      0.92     85803



In [22]:
# Melakukan import data test dan melakukan EDA
df_test = pd.read_csv("transact_class.txt", sep="|")

# Mengubah "?" menjadi not a number
df_test = df_test.replace('?', np.nan)

# Menambah data pada kolom yang merupakan tipe data float atau int menggunakan mean
num = df_test.select_dtypes(include=['float64', 'int64']).columns
num_imputer = SimpleImputer(strategy='mean')
df_test[num] = num_imputer.fit_transform(df_test[num])

# Menambah data pada kolom yang merupakan tipe data object menggunakan 'most_frequent'
obj = df_test.select_dtypes(include='object').columns
obj_imputer = SimpleImputer(strategy='most_frequent')
df_test[obj] = obj_imputer.fit_transform(df_test[obj])

# Menentukan feature
x = df_test.iloc[:, [2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 16, 18, 19, 20, 22]].values

# Melakukan scaling data
scaler = StandardScaler()
x_test = scaler.fit_transform(x)

# Melakukan prediksi menggunakan model yang telah ada
predicted_test = classi.predict(x_test)
# Menambahkan kolom 'predicted_order' pada dataframe 'df_test'
df_test['predicted_order'] = predicted_test

print(df_test.tail())
    

       sessionNo  startHour  startWeekday  duration  cCount cMinPrice  \
45063     5111.0        5.0           1.0    26.286     3.0      4.99   
45064     5111.0        5.0           1.0    30.059     3.0      4.99   
45065     5111.0        5.0           1.0    33.388     3.0      4.99   
45066     5111.0        5.0           1.0    51.641     3.0      4.99   
45067     5111.0        5.0           1.0    55.953     3.0      4.99   

      cMaxPrice cSumPrice  bCount bMinPrice  ...          availability  \
45063      4.99      9.98     1.0      4.99  ...  completely orderable   
45064      4.99      9.98     1.0      4.99  ...  completely orderable   
45065      4.99      9.98     1.0      4.99  ...  completely orderable   
45066      4.99      9.98     1.0      4.99  ...  completely orderable   
45067      4.99      9.98     1.0      4.99  ...  completely orderable   

      customerNo maxVal customerScore accountLifetime payments age address  \
45063         47    600            70 